In [1]:
%cd beta-tcvae
# %cd content

In [2]:
import os
import time
import math
from numbers import Number
import argparse

import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.autograd import Variable
from torchvision import datasets
import visdom
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import brewer2mpl
from tqdm import tqdm

import lib.dist as dist
import lib.utils as utils
import lib.datasets as dset
from lib.flows import FactorialNormalizingFlow

from importlib import reload

# from metric_helpers.loader import load_model_and_dataset

from plot_latent_vs_true import plot_vs_gt_shapes, plot_vs_gt_faces  # noqa: F401
from lib.model import *
from metric_helpers.elbo_decomposition import *
from metric_helpers.mi_metric import compute_metric_shapes, compute_metric_faces, compute_metric_mnist


import metric_helpers
reload(metric_helpers.elbo_decomposition)
reload(utils)
# reload(model)

<module 'lib.utils' from '/Users/visriv/Documents/Git/beta-tcvae/lib/utils.py'>

In [3]:
win_samples = None
win_test_reco = None
win_latent_walk = None
win_train_elbo = None



class Object(object):
    pass

args = Object()
args.dataset = 'mnist'
args.dist = 'normal'
args.num_epochs = 10
args.batch_size = 32
args.learning_rate = 1e-3
args.latent_dim = 10
args.beta = 1
args.tcvae = True
args.exclude_mutinfo = False
args.beta_anneal = False
args.lambda_anneal = False
args.mss = False
args.conv = True
args.gpu = 0
args.visdom = False
args.save = 'test'
args.log_freq = 100

In [4]:
# torch.cuda.set_device(args.gpu)
use_cuda = False
# data loader
train_loader = utils.setup_data_loaders(args, use_cuda=False)



# setup the VAE
if args.dist == 'normal':
    prior_dist = dist.Normal()
    q_dist = dist.Normal()
elif args.dist == 'laplace':
    prior_dist = dist.Laplace()
    q_dist = dist.Laplace()
elif args.dist == 'flow':
    prior_dist = FactorialNormalizingFlow(dim=args.latent_dim, nsteps=32)
    q_dist = dist.Normal()

vae = VAE(z_dim=args.latent_dim, use_cuda=use_cuda, prior_dist=prior_dist, q_dist=q_dist, image_dim = 28,
    include_mutinfo=not args.exclude_mutinfo, tcvae=args.tcvae, conv=args.conv, mss=args.mss)

# setup the optimizer
optimizer = optim.Adam(vae.parameters(), lr=args.learning_rate)

# setup visdom for visualization
if args.visdom:
    vis = visdom.Visdom(env=args.save, port=4500)

train_elbo = []

# training loop
dataset_size = len(train_loader.dataset)
num_iterations = len(train_loader) * args.num_epochs
iteration = 0
# initialize loss accumulator
elbo_running_mean = utils.RunningAverageMeter()
while iteration < num_iterations:
    for i, x in enumerate(train_loader):
        iteration += 1
        print(iteration)
        batch_time = time.time()
        vae.train()
        anneal_kl(args, vae, iteration)
        optimizer.zero_grad()
        # transfer to GPU
        x = x[0]
        # print(type(x))
        # print(x)
        # print(len(x), len(x[0]), len(x[0][0]), len(x[0][0][0]))
        if use_cuda:
            x = x.cuda()
        # wrap the mini-batch in a PyTorch Variable
        x = Variable(x)
        # do ELBO gradient and accumulate loss
        obj, elbo = vae.elbo(x, dataset_size, 28)
#         print('elbo', elbo)
        if utils.isnan(obj).any():
            raise ValueError('NaN spotted in objective.')
        obj.mean().mul(-1).backward()
        elbo_running_mean.update(torch.mean(elbo.mean()))
        optimizer.step()

        # report training diagnostics
        if iteration % args.log_freq == 0:
            train_elbo.append(elbo_running_mean.avg)
            print('[iteration %03d] time: %.2f \tbeta %.2f \tlambda %.2f training ELBO: %.4f (%.4f)' % (
                iteration, time.time() - batch_time, vae.beta, vae.lamb,
                elbo_running_mean.val, elbo_running_mean.avg))

            vae.eval()

            # plot training and test ELBOs
            if args.visdom:
                display_samples(vae, x, vis)
                plot_elbo(train_elbo, vis)

            utils.save_checkpoint({
                'state_dict': vae.state_dict(),
                'args': args}, args.save, 0)
            eval('plot_vs_gt_' + args.dataset)(vae, train_loader.dataset,
                                               os.path.join(args.save, 'gt_vs_latent_{:05d}.png'.format(iteration)),
                                               use_cuda=False)

In [5]:
# torch.cuda.set_device(args.gpu)
use_cuda = False
# data loader
train_loader = utils.setup_data_loaders(args, use_cuda=False)



# setup the VAE
if args.dist == 'normal':
    prior_dist = dist.Normal()
    q_dist = dist.Normal()
elif args.dist == 'laplace':
    prior_dist = dist.Laplace()
    q_dist = dist.Laplace()
elif args.dist == 'flow':
    prior_dist = FactorialNormalizingFlow(dim=args.latent_dim, nsteps=32)
    q_dist = dist.Normal()

vae = VAE(z_dim=args.latent_dim, use_cuda=use_cuda, prior_dist=prior_dist, q_dist=q_dist, image_dim = 28,
    include_mutinfo=not args.exclude_mutinfo, tcvae=args.tcvae, conv=args.conv, mss=args.mss)

# setup the optimizer
optimizer = optim.Adam(vae.parameters(), lr=args.learning_rate)

# setup visdom for visualization
if args.visdom:
    vis = visdom.Visdom(env=args.save, port=4500)

train_elbo = []

# training loop
dataset_size = len(train_loader.dataset)
num_iterations = len(train_loader) * args.num_epochs
iteration = 0
# initialize loss accumulator
elbo_running_mean = utils.RunningAverageMeter()
while iteration < num_iterations:
    for i, x in enumerate(train_loader):
        iteration += 1
        print(iteration)
        batch_time = time.time()
        vae.train()
        utils.anneal_kl(args, vae, iteration)
        optimizer.zero_grad()
        # transfer to GPU
        x = x[0]
        # print(type(x))
        # print(x)
        # print(len(x), len(x[0]), len(x[0][0]), len(x[0][0][0]))
        if use_cuda:
            x = x.cuda()
        # wrap the mini-batch in a PyTorch Variable
        x = Variable(x)
        # do ELBO gradient and accumulate loss
        obj, elbo = vae.elbo(x, dataset_size, 28)
#         print('elbo', elbo)
        if utils.isnan(obj).any():
            raise ValueError('NaN spotted in objective.')
        obj.mean().mul(-1).backward()
        elbo_running_mean.update(torch.mean(elbo.mean()))
        optimizer.step()

        # report training diagnostics
        if iteration % args.log_freq == 0:
            train_elbo.append(elbo_running_mean.avg)
            print('[iteration %03d] time: %.2f \tbeta %.2f \tlambda %.2f training ELBO: %.4f (%.4f)' % (
                iteration, time.time() - batch_time, vae.beta, vae.lamb,
                elbo_running_mean.val, elbo_running_mean.avg))

            vae.eval()

            # plot training and test ELBOs
            if args.visdom:
                display_samples(vae, x, vis)
                plot_elbo(train_elbo, vis)

            utils.save_checkpoint({
                'state_dict': vae.state_dict(),
                'args': args}, args.save, 0)
            eval('plot_vs_gt_' + args.dataset)(vae, train_loader.dataset,
                                               os.path.join(args.save, 'gt_vs_latent_{:05d}.png'.format(iteration)),
                                               use_cuda=False)

In [6]:
import os
import time
import math
from numbers import Number
import argparse

import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.autograd import Variable
from torchvision import datasets
import visdom
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import brewer2mpl
from tqdm import tqdm

import lib.dist as dist
import lib.utils as utils
import lib.datasets as dset
from lib.flows import FactorialNormalizingFlow

from importlib import reload

# from metric_helpers.loader import load_model_and_dataset

from plot_latent_vs_true import plot_vs_gt_shapes, plot_vs_gt_faces, plot_vs_gt_mnist # noqa: F401
from lib.model import *
from metric_helpers.elbo_decomposition import *
from metric_helpers.mi_metric import compute_metric_shapes, compute_metric_faces, compute_metric_mnist


import metric_helpers
reload(metric_helpers.elbo_decomposition)
reload(utils)
# reload(model)

<module 'lib.utils' from '/Users/visriv/Documents/Git/beta-tcvae/lib/utils.py'>

In [7]:
# torch.cuda.set_device(args.gpu)
use_cuda = False
# data loader
train_loader = utils.setup_data_loaders(args, use_cuda=False)



# setup the VAE
if args.dist == 'normal':
    prior_dist = dist.Normal()
    q_dist = dist.Normal()
elif args.dist == 'laplace':
    prior_dist = dist.Laplace()
    q_dist = dist.Laplace()
elif args.dist == 'flow':
    prior_dist = FactorialNormalizingFlow(dim=args.latent_dim, nsteps=32)
    q_dist = dist.Normal()

vae = VAE(z_dim=args.latent_dim, use_cuda=use_cuda, prior_dist=prior_dist, q_dist=q_dist, image_dim = 28,
    include_mutinfo=not args.exclude_mutinfo, tcvae=args.tcvae, conv=args.conv, mss=args.mss)

# setup the optimizer
optimizer = optim.Adam(vae.parameters(), lr=args.learning_rate)

# setup visdom for visualization
if args.visdom:
    vis = visdom.Visdom(env=args.save, port=4500)

train_elbo = []

# training loop
dataset_size = len(train_loader.dataset)
num_iterations = len(train_loader) * args.num_epochs
iteration = 0
# initialize loss accumulator
elbo_running_mean = utils.RunningAverageMeter()
while iteration < num_iterations:
    for i, x in enumerate(train_loader):
        iteration += 1
        print(iteration)
        batch_time = time.time()
        vae.train()
        utils.anneal_kl(args, vae, iteration)
        optimizer.zero_grad()
        # transfer to GPU
        x = x[0]
        # print(type(x))
        # print(x)
        # print(len(x), len(x[0]), len(x[0][0]), len(x[0][0][0]))
        if use_cuda:
            x = x.cuda()
        # wrap the mini-batch in a PyTorch Variable
        x = Variable(x)
        # do ELBO gradient and accumulate loss
        obj, elbo = vae.elbo(x, dataset_size, 28)
#         print('elbo', elbo)
        if utils.isnan(obj).any():
            raise ValueError('NaN spotted in objective.')
        obj.mean().mul(-1).backward()
        elbo_running_mean.update(torch.mean(elbo.mean()))
        optimizer.step()

        # report training diagnostics
        if iteration % args.log_freq == 0:
            train_elbo.append(elbo_running_mean.avg)
            print('[iteration %03d] time: %.2f \tbeta %.2f \tlambda %.2f training ELBO: %.4f (%.4f)' % (
                iteration, time.time() - batch_time, vae.beta, vae.lamb,
                elbo_running_mean.val, elbo_running_mean.avg))

            vae.eval()

            # plot training and test ELBOs
            if args.visdom:
                display_samples(vae, x, vis)
                plot_elbo(train_elbo, vis)

            utils.save_checkpoint({
                'state_dict': vae.state_dict(),
                'args': args}, args.save, 0)
            eval('plot_vs_gt_' + args.dataset)(vae, train_loader.dataset,
                                               os.path.join(args.save, 'gt_vs_latent_{:05d}.png'.format(iteration)),
                                               use_cuda=False)

In [8]:
import wandb
wandb.login()

True

In [9]:
wandb.init(project="my-awesome-project")

In [10]:
wandb.init(project="my-awesome-project")